# Code for Invenstment under Taxation

### LP formulation for the problem:
\begin{equation}
\begin{array}{ll}
\nonumber
\max &  \sum_{i=1}^{n}r_i(s_i-x_i)\\
s.t. &  \sum_{i=1}^{n}(q_i x_i - 0.3\max\{q_i-p_i,0\}x_i-0.01x_iq_i) \ge K\\
 & 0\le x_i\le s_i, \forall i = 1,...,n
\end{array}
\end{equation}

In [1]:
from gurobipy import *
import numpy as np
import pandas as pd


#########Parameters Set-up############

K = 9000

# read data 
invest_data = pd.read_csv('./investment.csv')
print(invest_data)
invest_data = invest_data.values
print(invest_data)

  Stock  Buying Price  Current Share  Current Price  Expected Future Price
0    S1           1.2           1000            2.1                    2.0
1    S2           2.1           1000            3.2                    3.7
2    S3           3.2           1000            4.1                    5.2
3    S4           4.1           1000            5.1                    7.1
4    S5           4.5           1000            6.7                    9.1
[['S1' 1.2 1000 2.1 2.0]
 ['S2' 2.1 1000 3.2 3.7]
 ['S3' 3.2 1000 4.1 5.2]
 ['S4' 4.1 1000 5.1 7.1]
 ['S5' 4.5 1000 6.7 9.1]]


In [2]:
#define all the parameters as a dictionary with stock being the index
stock, buy_price, cur_share, cur_price, exp_price = multidict({item[0]: (item[1], item[2], item[3], item[4]) 
                                                              for item in invest_data })

print(stock, buy_price)

['S1', 'S2', 'S3', 'S4', 'S5'] {'S1': 1.2, 'S2': 2.1, 'S3': 3.2, 'S4': 4.1, 'S5': 4.5}


In [3]:
#########Model Set-up############

m = Model("invest_tax")

# number of shares of stock i to be sold
sell = m.addVars(stock, ub = cur_share, lb=0, name = "sell")

# first define objective
obj = quicksum( exp_price[i] * (cur_share[i] - sell[i]) for i in stock)

# then add objective
m.setObjective( obj, GRB.MAXIMIZE)

# first define the constraint that requries at least K needs to be raised
C_min_raise =  quicksum(sell[i]*cur_price[i] - 0.3*sell[i]*( max(cur_price[i]-buy_price[i],0)) - 0.01*sell[i]*cur_price[i] 
                       for i in stock) >= K

# then add the constraint
m.addConstr(C_min_raise, "min_raise")

# Alternatively one can combine the two steps as we did in the class
# m.addConstr( quicksum(sell[i]*cur_price[i] - 0.3*sell[i]*( max(cur_price[i]-buy_price[i],0)) - 0.01*sell[i]*cur_price[i] 
#                        for i in stock) >= K, "min_raise")


m.optimize()

#  Print optimal solutions and optimal value
print("\n Optimal solution:")
for v in m.getVars():
    print(v.VarName, v.x)

print("\n Optimal profit:")
print('Obj:', m.objVal)

Academic license - for non-commercial use only
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [2e+00, 6e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [1e+03, 1e+03]
  RHS range        [9e+03, 9e+03]
Presolve time: 0.00s
Presolved: 1 rows, 5 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7100000e+04   5.625000e+02   0.000000e+00      0s
       1    1.5356791e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  1.535679090e+04

 Optimal solution:
sell[S1] 1000.0
sell[S2] 1000.0
sell[S3] 1000.0
sell[S4] 118.76184459886302
sell[S5] 0.0

 Optimal profit:
Obj: 15356.790903348072
